In [ ]:
from transformers import WhisperFeatureExtractor
import librosa
import matplotlib.pyplot as plt
import subprocess
import os
import torch


In [ ]:
def convert_opus_to_wav(opus_path, wav_path, path_to_ffmpeg):
    '''This function converts an opus file to a wav file.
    opus_path: path to the opus file
    wav_path: path to the wav file
    path_to_ffmpeg: path to the ffmpeg executable'''
    # print('Converting opus to wav')

    # result = subprocess.run([f'{path_to_ffmpeg}', '-i', f'{opus_file}', '-vn', f'{wav_file}'])
    result = subprocess.run([f'{path_to_ffmpeg}', '-i', f'{opus_path}', '-vn', f'{wav_path}'])
    # print('result')

    # Check the exit code
    if result.returncode == 0:
        print('Conversion successful')
    else:
        print('Conversion failed')

In [ ]:
def convert_opus_to_wav_in_dir(opus_dir, wav_dir, path_to_ffmpeg):
    '''This function converts all opus files in a directory to wav files.
    opus_dir: path to the directory with opus files
    wav_dir: path to where the wav files should be stored
    path_to_ffmpeg: path to the ffmpeg executable'''

    # go through all the folders and subfolders and get all the opus files
    opus_files = []
    for root, dirs, files in os.walk(opus_dir):
        for file in files:
            if file.endswith('.opus'):
                opus_files.append(os.path.join(root, file))

    print(f"{len(opus_files)} opus_fiels created")

    # for each opus file, convert it to a wav file
    for opus_file in opus_files:
        # get the name of the wav file
        wav_file = opus_file.replace(opus_dir, wav_dir)
        wav_file = wav_file.replace('.opus', '.wav')

        # create the directory for the wav file
        os.makedirs(os.path.dirname(wav_file), exist_ok=True)

        # convert the opus file to a wav file
        convert_opus_to_wav(opus_file, wav_file, path_to_ffmpeg)
    print('Conversions done')

In [ ]:
opus_dir = 'D:\Documents\Data_Science\MDS_CL\Term2\Block6\COLX_585_trends_in_computational_linguistics\Team_project\data\\testing_func_opus'
wav_dir = 'D:\Documents\Data_Science\MDS_CL\Term2\Block6\COLX_585_trends_in_computational_linguistics\Team_project\data\\testing_func_opus_wav'
path_to_ffmpeg = "C:\Program Files\\ffmpeg\\bin/ffmpeg.exe"
convert_opus_to_wav_in_dir(opus_dir, wav_dir, path_to_ffmpeg)

In [ ]:
def generate_spectrogram(file, size):
    '''This function takes in a file path and a size and returns a spectrogram of the audio file.
    file: path to the audio file (wav)
    size: a tuple, size of the spectrogram (height, width)'''
    audio, sr = librosa.load(file, sr=16000) # load audio file
    feature_extractor = WhisperFeatureExtractor() # initialize feature extractor
    features = feature_extractor(audio, sampling_rate=sr, return_tensors="pt") # extract features
    spectrogram = features['input_features'][0] # get spectrogram
    spectrogram = spectrogram[:size[0], :size[1]] # crop spectrogram
    return spectrogram

In [ ]:
def generate_spectrograms_in_dir(wav_dir, spectrogram_dir, size):
    '''This function generates spectrograms for all wav files in a directory.
    wav_dir: path to the directory with wav files
    spectrogram_dir: path to the directory where the spectrograms will be saved
    size: a tuple, size of the spectrograms (height, width)'''

    wav_files = []
    for root, dirs, files in os.walk(wav_dir):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))

    # for each wav file, generate a spectrogram
    for wav_file in wav_files:
        # get the name of the spectrogram file
        spectrogram_file = wav_file.replace(wav_dir, spectrogram_dir)

        spectrogram_file = spectrogram_file.replace('.wav', '.pt')

        # create the directory for the spectrogram file
        os.makedirs(os.path.dirname(spectrogram_file), exist_ok=True)

        # generate the spectrogram
        spectrogram = generate_spectrogram(wav_file, size)

        # save the spectrogram
        torch.save(spectrogram, spectrogram_file)

    print('Spectrograms done')

In [ ]:
# test the function
wav_dir = 'D:\Documents\Data_Science\MDS_CL\Term2\Block6\COLX_585_trends_in_computational_linguistics\Team_project\data\\testing_func_opus_wav'
spectrogram_dir = 'D:\Documents\Data_Science\MDS_CL\Term2\Block6\COLX_585_trends_in_computational_linguistics\Team_project\data\\testing_func_opus_wav_spectrogram'
size = (49, 40)
generate_spectrograms_in_dir(wav_dir, spectrogram_dir, size)